# Launching AI agents from PoC to Production in just a few lines of code

Strands Agents is a simple yet powerful SDK that takes a model-driven approach to building and running AI agents. From simple conversational assistants to complex autonomous workflows, from local development to production deployment, Strands Agents scales with your needs.<br/><br/>
This notebook demonstrates how to leverage the **Strands Agents SDK and Tools** to generate and execute automated Python code across a diverse range of tasks.

### Install Required Packages
Installs `strands-agents` and related tools for creating Python agents and tools that can run and evaluate Python code.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Install required packages with tested versions
!pip install -r requirements.txt -U

# Note: bedrock-agentcore-starter-toolkit is now in requirements.txt with correct version (0.1.5)
# No need for separate installation

from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

### Import Required Packages and Configure Environment

In [ ]:
import os
import json
import time
import requests
from datetime import datetime
from pathlib import Path

# Strands imports
from strands.models.bedrock import BedrockModel
from strands import Agent, tool
from strands_tools import retrieve, http_request
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.stdio import stdio_client, StdioServerParameters

# AgentCore imports
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore_starter_toolkit import Runtime

from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

# Configure environment
os.environ["BYPASS_TOOL_CONSENT"] = "true"

# Configure model
model = BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0")

print("All imports loaded successfully")
print(region)

### Create Bedrock Knowledge Base

In [ ]:
!python utils/create_kb.py

### Configure the Model for Strands Agents

<div class="alert alert-block alert-info">
	⚠️ <b>Important:</b> ⚠️</br> To use <b>Amazon SageMaker AI</b>, make sure you've deployed the model according to <code>0-setup/2-setup-sagemaker-endpoint.ipynb</code> before proceeding.
</div>

In [ ]:
# %store -r SAGEMAKER_ENDPOINT_NAME
# print(f"Endpoint name: {SAGEMAKER_ENDPOINT_NAME}")

In [2]:
from strands.models.sagemaker import SageMakerAIModel
from strands.models.bedrock import BedrockModel


provider = "BEDROCK"  # Change this to SAGEMAKER to use the previously deployed endpoint instead of Bedrock

match provider:
    case "BEDROCK":
        # Using Claude 3.5 Sonnet from Bedrock
        model = BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0")
    case "SAGEMAKER":
        pass
        # Using Qwen3 from our endpoint in SageMaker AI
        model = SageMakerAIModel(
			endpoint_config={"endpoint_name": SAGEMAKER_ENDPOINT_NAME},
			payload_config={"max_tokens": 1024*5, "stream": True}
		)

# Agent 1: Web Research Agent


In [ ]:
def create_web_research_agent():
    """Create a real web research agent with actual scraping capabilities"""
    system_prompt = """
    You are a web research specialist. Your capabilities include:
    
    1. Web Scraping: Use requests and BeautifulSoup to scrape web content
    2. Data Extraction: Extract titles, links, text content, and structured data
    3. Content Analysis: Analyze and summarize web content
    
    Always use proper web scraping practices and extract meaningful, structured information.
    """
    
    return Agent(
        model=model,
        tools=[http_request],
        system_prompt=system_prompt
    )

# Create the web research agent
web_agent = create_web_research_agent()

print("Created Web Research Agent")

Test Web Research Agent

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")

web_test_query = f"""
Scrape the Hacker News front page (https://news.ycombinator.com/) and extract:
1. The top 10 article titles
2. Their corresponding URLs
3. Points/score if available
4. Number of comments if available

Provide a brief summary of the main topics being discussed.
"""

print("Testing Web Research Agent with real scraping...")
web_result = web_agent(web_test_query)
print("Web scraping test completed")


# Agent 2: Knowledge Base Agent

Extract Knowledge Base ID

In [ ]:
# Load the knowledge base ID
with open('kb_id.txt', 'r') as f:
    kb_id = f.read().strip()
    
print(f"Knowledge Base ID: {kb_id}")

Create Knowledge Base Agent

In [ ]:
def create_kb_agent(kb_id: str):
    """Create a real web research agent with actual scraping capabilities"""

    system_prompt = f""" 
    You are connected to knowledge base {kb_id} in {region}. 
    Search the knowledge base for information to answer users questions.
    Be specific about what you find.
    """

    return Agent(
        model=model,
        tools=[retrieve],
        system_prompt=system_prompt
    )

# Create the web research agent
kb_agent = create_kb_agent(kb_id)

print("Created Knowledge Base Agent")

Test Knowledge Base Agent

In [ ]:
kb_test_query = f"""
What is Agentcore?
"""

print("Testing Knowledge Base Agent with real scraping...")
web_result = kb_agent(kb_test_query)
print("Knowledge Base test completed")


# Productionize the Agents with Amazon Bedrock Agentcore

Create IAM Role for Deployed Agents

In [4]:
import boto3
import json

def create_agentcore_role(kb_id):
    iam = boto3.client('iam')
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    policy = {
        "Version": "2012-10-17",
        "Statement": [
            {"Effect": "Allow", "Action": ["bedrock:InvokeModel", "bedrock:InvokeModelWithResponseStream"], "Resource": "arn:aws:bedrock:*::foundation-model/*"},
            {"Effect": "Allow", "Action": ["bedrock:InvokeModel", "bedrock:InvokeModelWithResponseStream"], "Resource": f"arn:aws:bedrock:*:{account_id}:inference-profile/*"},
            {"Effect": "Allow", "Action": ["bedrock:Retrieve", "bedrock:RetrieveAndGenerate"], "Resource": f"arn:aws:bedrock:us-east-1:{account_id}:knowledge-base/*"},
            {"Effect": "Allow", "Action": ["ecr:BatchGetImage", "ecr:GetDownloadUrlForLayer"], "Resource": f"arn:aws:ecr:us-east-1:{account_id}:repository/*"},
            {"Effect": "Allow", "Action": ["ecr:GetAuthorizationToken"], "Resource": "*"},
            {"Effect": "Allow", "Action": ["logs:CreateLogGroup", "logs:CreateLogStream", "logs:PutLogEvents"], "Resource": f"arn:aws:logs:us-east-1:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*"},
            {"Effect": "Allow", "Action": ["cloudwatch:PutMetricData"], "Resource": "*"},
            {"Effect": "Allow", "Action": ["xray:PutTraceSegments", "xray:PutTelemetryRecords"], "Resource": "*"},
            {"Effect": "Allow", "Action": ["bedrock-agentcore:GetWorkloadAccessToken", "bedrock-agentcore:GetWorkloadAccessTokenForJWT", "bedrock-agentcore:GetWorkloadAccessTokenForUserId"], "Resource": [f"arn:aws:bedrock-agentcore:us-east-1:{account_id}:workload-identity-directory/default", f"arn:aws:bedrock-agentcore:us-east-1:{account_id}:workload-identity-directory/default/workload-identity/*"]}
        ]
    }
    
    try:
        iam.create_role(RoleName="AgentCoreRuntimeRole", AssumeRolePolicyDocument=json.dumps(trust_policy))
        iam.put_role_policy(RoleName="AgentCoreRuntimeRole", PolicyName="Policy", PolicyDocument=json.dumps(policy))
    except: pass
    
    return f"arn:aws:iam::{account_id}:role/AgentCoreRuntimeRole"

agentcore_role_arn = create_agentcore_role(kb_id)

Helper Function to launch Agents to AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import json
import time

def deploy_agent_to_agentcore(agent_file, agent_name):
    """Deploy agent using Python SDK (more reliable for notebooks)"""
    print(f"Deploying {agent_name} using Python SDK...")
    
    try:
        # Initialize runtime
        runtime = Runtime()
        
        # Configure
        print("Configuring agent...")
        runtime.configure(
            entrypoint=agent_file,
            agent_name=agent_name,
            execution_role=agentcore_role_arn,
            auto_create_ecr=True,
            requirements_file="requirements.txt"
        )
        print("Configuration successful!")
        
        # Launch with CodeBuild
        print("Launching with CodeBuild...")
        runtime.launch(auto_update_on_conflict=True)
        print("Launch successful!")
        
        return runtime
        
    except Exception as e:
        print(f"Deployment failed: {str(e)}")
        print(f"Error type: {type(e).__name__}")
        
        # Print the full traceback for debugging
        import traceback
        print("Full traceback:")
        traceback.print_exc()
        
        return None

print("Python SDK deployment function ready!")

Deploy Web Agent

In [ ]:
# Create agent file using Python file operations
agent_code = '''from bedrock_agentcore import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands_tools import http_request

app = BedrockAgentCoreApp()

# Create the web research agent with scraping capabilities
agent = Agent(
    model=BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0"),
    tools=[http_request],
    system_prompt="""
    You are a web research specialist. Your capabilities include:
    
    1. Web Scraping: Use requests and BeautifulSoup to scrape web content
    2. Data Extraction: Extract titles, links, text content, and structured data
    3. Content Analysis: Analyze and summarize web content
    
    Always use proper web scraping practices and extract meaningful, structured information.
    """
)

@app.entrypoint
def agent_entrypoint(payload):
    """
    Invoke the agent with the payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()'''

# Write the agent file
with open('web_research_agent.py', 'w') as f:
    f.write(agent_code)

print("web_research_agent.py created successfully!")

# Deploy using Runtime class (CodeBuild should bypass Docker)
web_runtime = deploy_agent_to_agentcore(
    agent_file="web_research_agent.py",
    agent_name="web_research_agent"
)

Test Deployed Web Agent

In [ ]:
invoke_response = web_runtime.invoke({"prompt": f"""
Scrape the Hacker News front page (https://news.ycombinator.com/) and extract:
1. The top 10 article titles
2. Their corresponding URLs
3. Points/score if available
4. Number of comments if available

Provide a brief summary of the main topics being discussed.
"""})
invoke_response

Deploy KB Agent

In [ ]:
# Create Knowledge Base Agent following the official amazon-bedrock-agentcore-samples pattern
print("Creating Knowledge Base agent with correct AgentCore pattern...")

kb_agent_code = f'''import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"
os.environ["KNOWLEDGE_BASE_ID"] = "{kb_id}"
os.environ["DEFAULT_KNOWLEDGE_BASE_ID"] = "{kb_id}"
os.environ["AWS_REGION"] = "{region}"

from strands import Agent
from strands_tools import retrieve
from strands.models import BedrockModel

agent = Agent(
    model=BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0"),
    tools=[retrieve],
    system_prompt="""You are connected to knowledge base {kb_id} in {region}. 
    Search the knowledge base for information to answer users questions.
    Be specific about what you find."""
)

from bedrock_agentcore.runtime import BedrockAgentCoreApp
app = BedrockAgentCoreApp()

@app.entrypoint
def agent_invocation(payload, context):
    """Handler for agent invocation"""
    user_message = payload.get("prompt", "Hello! How can I help you search the knowledge base?")
    result = agent(user_message)
    print("context:\\n-------\\n", context)
    print("result:\\n*******\\n", result)
    return {{"result": result.message}}

if __name__ == "__main__":
    app.run()'''

# Write the KB agent file
with open('kb_agent.py', 'w') as f:
    f.write(kb_agent_code)

print("kb_agent.py created successfully using official AgentCore samples pattern!")
print(f"Knowledge Base ID configured: {kb_id}")

# Deploy the Knowledge Base agent
print("Deploying Knowledge Base agent to AgentCore...")
kb_runtime = deploy_agent_to_agentcore(
    agent_file="kb_agent.py",
    agent_name="kb_agent"
)

Test Deployed Knowledge Base Agent

In [ ]:
invoke_response = kb_runtime.invoke({"prompt": f"""
What is AgentCore?
"""})
invoke_response

# Create an Orchestrator Agent to leverage deployed Agents

Create Local Orchestrator Agent

In [ ]:
# Get agent ARNs from deployed agents
kb_agent_arn = kb_runtime.status().config.agent_arn
web_agent_arn = web_runtime.status().config.agent_arn

print(f"KB Agent ARN: {kb_agent_arn}")
print(f"Web Agent ARN: {web_agent_arn}")

# Create AgentCore client
agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

def invoke_agentcore_agent(agent_arn: str, query: str, agent_type: str = "agent") -> str:
    """Helper function to invoke AgentCore agents with consistent error handling"""
    try:
        response = agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            qualifier="DEFAULT",
            payload=json.dumps({"prompt": query}),
            contentType="application/json",
            accept="application/json"
        )
        
        content = ''.join(chunk.decode('utf-8') for chunk in response.get("response", []))
        result = json.loads(content)
        if isinstance(result, dict) and 'result' in result:
            result_data = result['result']
            
            if isinstance(result_data, dict) and 'content' in result_data:
                if isinstance(result_data['content'], list):
                    # Extract text from content array
                    text_parts = []
                    for item in result_data['content']:
                        if isinstance(item, dict) and 'text' in item:
                            text_parts.append(item['text'])
                    return ''.join(text_parts)
                else:
                    return str(result_data['content'])
            else:
                return str(result_data)
        else:
            return str(result)
        
    except Exception as e:
        return f"Error searching {agent_type}: {str(e)}"

@tool
def search_knowledge_base(query: str) -> str:
    """Search the internal knowledge base for information about AgentCore, AWS services, and technical documentation.
    
    Args:
        query: The search query for the knowledge base
        
    Returns:
        str: Information retrieved from the knowledge base
    """
    return invoke_agentcore_agent(kb_agent_arn, f"{query}", "knowledge base")

@tool
def search_web(query: str) -> str:
    """Search the web for current information, trends, and real-time data.
    
    Args:
        query: The web search query
        
    Returns:
        str: Information retrieved from web sources
    """
    return invoke_agentcore_agent(web_agent_arn, f"{query}", "web search")

# Create the research agent
research_agent = Agent(
    model=model,
    tools=[search_knowledge_base, search_web],
    system_prompt="""You are an intelligent research agent that helps users find and correlate information from multiple sources.

You have access to two specialized tools:
1. search_knowledge_base: For internal documentation, AWS services info, and technical knowledge
2. search_web: For current information, trends, and real-time data

Research Strategy:
- Always synthesize information from both sources using search_knowledge_base and search_web tools
- Provide clear, well-structured responses with source attribution (URLs where available)
- Always add citations to your answers

Be thorough in your research and provide detailed, accurate answers."""
)

print("\nResearch Agent created successfully!")
print("Available tools: search_knowledge_base, search_web")
print("\nExample usage:")
print("result = research_agent('What is AgentCore and how does it compare to current AI agent platforms?')")
print("print(result.message)")

Test Orchestrator Agent

In [ ]:
research_agent('What is AgentCore and how does it compare to current AI agent platforms?')